
<div align="center">
  <a href="https://www.w-hs.de/maschinenbau-master-boh/">
    <img src="https://www.w-hs.de/typo3conf/ext/whs/Resources/Public/Images/Pagelayout/w-hs_pagelogo.png" 
    alt="Images" width="500" height="200">
  </a>
</div>
<br>
<h1 align="center"> Sondergebiete der Simulation</h1>
<h3 align="center"> WS 21/22 </h3>
<br />

## Workflow


> * Symbolische Herleitung der Bewegungsgleichung des freien, gedämpften Inversen Pendels mit **`sympy`**.
> * Numerische Lösung der DGL mit Hilfe von **`scipy.itegrate.odeint`**.
> * Trainingspunkte, die eine reale Messung ersetzen, aus numerischer Lösung extrahieren.
> * Interpolation der Punkte mit Hilfe eines *Neuronales Netzes.
> * Training eines PINNs, um den Bewegungsablauf ausßerhalb der Trainingspunkte anzunähern, indem DGL in die Loss-Funktion eingebettet wird.


### Environment set up

Sollten Fehlermeldungen erscheinen, obwohl alle benötigten Module installiert sind, hilft es wohlmöglich eine virtuelle Umgebung für das Arbeiten mit PINNs einzurichten:
```bash
conda create -n pinn python=3
conda activate pinn
conda install jupyter numpy matplotlib
conda install pytorch torchvision torchaudio -c pytorch
conda install scipy, sympy
```

#### Activate Enviroment
```bash
conda activate pinn
jupyter notebook 
```
<br />
<br />

# Credits: [benmoseley](https://github.com/benmoseley/harmonic-oscillator-pinn)

<br />
<br />

In [ ]:
import sympy
from sympy import symbols, Function, diff, sin, cos, Matrix, Rational, Eq, solve, lambdify
import sympy.physics.mechanics as mech
mech.init_vprinting()

### Herleitung der Bewegungsgleichungen 

Ausführliche Beschreibung der Herleitung:
[Inverted Pendulum Legrange](https://github.com/lennart2810/InvertedPendulumSDS/blob/master/MKS/Inverted%20Pendulum%20Legrange.ipynb) 

In [ ]:
# symbolischen Variablen anlegen
t, l, M, m, g, D, d, F = symbols('t l M m g, D, d, F')

# Zwangsbedingung
y = 0

# generalisierte Koordinaten und deren Ableitungen:
x = Function('x')(t)
x_d = diff(x,t)
x_dd = diff(x_d,t)

theta = Function('theta')(t)
theta_d = diff(theta,t)
theta_dd = diff(theta,t,t)

# Kinematik
x2, y2 = x - l * sin(theta), y + l * cos(theta)
x2_d, y2_d = x2.diff(t), y2.diff(t)

# Legrange-Funktion
T = Rational(1,2) * M * (x_d)**2 + Rational(1,2) * m * (x2_d**2 + y2_d**2)
V = M * g * y + m * g * y2 
L = T - V

# Euler-Legrange
LE_x = diff(diff(L, x_d), t) - diff(L, x)
LE_x = Eq(LE_x, F -  x_d * D)
LE_theta = diff(diff(L, theta_d), t) - diff(L, theta)
LE_theta = Eq(LE_theta, -theta_d * d)
display(Matrix([[LE_x.simplify()], [LE_theta.simplify()]]))

# Nach x_dd und theta_dd umstellen
solutions = solve([LE_x, LE_theta], (x_dd, theta_dd), simplify=True)

In [ ]:
# 0 = D*x_d + M*x_dd + m*(l*sin(theta)*theta_d**2-l*cos(theta)*theta_dd+x_dd)

# 0 = d*theta_d + l*m*(-g*sin(theta)+l*theta_dd-cos(theta)*x_dd)

### Umwandlung des symbolischen Gleichungssystems in numerische Funktionen mit `sympy.lambdify`.

In [ ]:
dxdt = lambdify(x_d, x_d)
dvdt = lambdify((t,g,M,m,l,x_d,theta,theta_d,F,D,d), solutions[x_dd])

dthetadt = lambdify(theta_d, theta_d)
domegadt = lambdify((t,g,M,m,l,x_d,theta,theta_d,F,D,d), solutions[theta_dd])

def dSdt(S, t, M, m, D, d, l, F):
    _x, _v, _theta, _omega = S
    return [
        dxdt(_v),
        dvdt(t, g, M, m, l, _v, _theta, _omega, F, D, d),
        dthetadt(_omega),
        domegadt(t, g, M, m , l, _v, _theta, _omega, F, D, d)
    ]

### Daten generieren

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.interpolate import interpolate

t1 = 10                          # s 
samples = 300                    # n 
t = np.linspace(0, t1, samples)  # s

g = 9.81                         # m/s^2
M = 5                            # kg
m = 1                            # kg
l = 1                            # m
F = 0                            # N
D = 0.5                          # N*s/m
d = 0.1                          # Nm*s

S0 = [0, 0, 0.1, 0]              # m, m/s, rad, rad/s

# numerische Lösung
ans = odeint(dSdt, y0=S0, t=t, args=(M, m, D, d, l, F))
x = ans.T[0]
theta = ans.T[2]

# numerische Lösung --> Lösungsfunktion interpolieren (für DeepXDE)
t_new = np.linspace(0, t1, samples+2000)    
x_f = interpolate.interp1d(t, x, 'cubic')
theta_f = interpolate.interp1d(t, theta, 'cubic')

# 'Messwerte'
a, b, i = 10, 180, 3 # jeden i. Punkt im Intervall [a,b] 
t_data = t[a:b:i]
x_data = x[a:b:i]
theta_data = theta[a:b:i]

# Data für NN zusammenfassen: (nur nötig wenn 1 Model mit 2 Outputs genutzt wird)
data = np.concatenate((x_data, theta_data)).reshape(2, len(t_data)).T
print('time.shape:', t_data.shape, type(t_data))
print('x.shape:', x_data.shape)
print('theta.shape:', theta_data.shape)
print('data.shape:', data.shape)


# Visualisierung 
plt.plot(t, x, 'b.')
plt.plot(t_new, x_f(t_new), 'g-')
plt.plot(t_data, data[:,0], 'ro') # data[:,1] --> x_data
plt.show()

plt.plot(t, theta, 'r.')
plt.plot(t_new, theta_f(t_new), 'y-')
plt.plot(t_data, data[:,1], 'go') # data[:,1] --> theta_data
plt.show()

In [ ]:
import torch

t = torch.Tensor(t).view(-1,1)
t_data = torch.Tensor(t_data).view(-1,1)
x_data = torch.Tensor(x_data).view(-1,1)
theta_data = torch.Tensor(theta_data).view(-1,1)
data = torch.Tensor(data).view(-1,2)

print('t:', t.shape)
print('t_data:', t_data.shape)
print('x_data:', x_data.shape)
print('theta_data:', theta_data.shape)
print('data:', data.shape)

In [ ]:
from PIL import Image

def save_gif_PIL(outfile, files, fps=5, loop=0):
    "Helper function for saving GIFs"
    imgs = [Image.open(file) for file in files]
    imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:], save_all=True, duration=int(1000/fps), loop=loop)

    
def plot_my_result(t, x, theta, t_data, data, pred, ps=None):
    
    fig = plt.figure(figsize=(16,8))
    ax = fig.add_subplot(111)
    fig.suptitle(name_plot, fontsize=10)
    
    # wahren Werte
    plt.plot(t, x, 'k--', linewidth=2, alpha=0.8, label="numerical solution $x$")
    plt.plot(t, theta, 'b--', linewidth=2, alpha=0.8, label=r"numerical solution $\theta$")
    
    # Messwerte
    plt.scatter(t_data, data[:,0], s=30, color="grey", label='Training data $x$')
    plt.scatter(t_data, data[:,1], s=30, color="royalblue", label=r'Training data $\theta$')
    
    # Zeitpunkte der Physik Loss berechnung:
    if ps is not None:
        plt.scatter(ps, -3*torch.ones_like(ps), s=20, color="limegreen", alpha=0.4, label='physics trainings')
    
    # Prediction
    plt.plot(t, pred[:,0], color="black", linewidth=2, alpha=0.8, label="Prediction $x$")
    plt.plot(t, pred[:,1], color="blue", linewidth=2, alpha=0.8, label=r"Prediction $\theta$")
    
    plt.text(torch.max(t)*1.01,torch.max(data),"episode: %i"%(i),fontsize=16,color="k")
    
    
    x_max = torch.max(t)
    y_min = torch.min(pred[:,1])
    y_max = torch.max(pred[:,1])
    
    plt.xlim(0, t1)
    plt.ylim(-1, 6.3)
    
    l = plt.legend(loc=(1.01,0.34), frameon=False, fontsize="large")
    plt.setp(l.get_texts(), color="k")
    #ax.get_yaxis().set_ticks([])
    plt.axis("off")

In [ ]:
## Classe eines Neuronalen Netzes erstellen, (mit einer forward Methode --> predict)

In [ ]:
import torch.nn as nn

class FCN(nn.Module):
    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
        
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

## Normal neural network

> Next, we train a standard neural network (fully connected) to fit these training points.

>We find that the network is able to fit the solution very closely in the vicinity of the training points, but does not learn an accurate solution outside of them.

>"One popular way of doing this using machine learning is to use a neural network. Given the location of a data point as input (denoted x), a neural network can be used to output a prediction of its value (denoted u), as shown in the figure below." [Ben Moseley](https://benmoseley.blog/my-research/so-what-is-a-physics-informed-neural-network/)

<div align="center">
  <a href="https://benmoseley.blog/my-research/so-what-is-a-physics-informed-neural-network/">
    <img src="https://benmoseley.blog/wp-content/uploads/2021/08/nn.png" 
    alt="Images" width="600">
  </a>
</div>

> To learn a model, we try to tune the network’s free parameters (denoted by the \thetas in the figure above) so that the network’s predictions closely match the available experimental data. This is usually done by minimising the mean-squared-error between its predictions and the training points;

>The problem is, using a purely data-driven approach like this can have significant downsides. Have a look at the actual values of the unknown physical process used to generate the experimental data in the animation above (grey line).

>You can see that whilst the neural network accurately models the physical process within the vicinity of the experimental data, it fails to generalise away from this training data. By only relying on the data, one could argue it hasn’t truly “understood” the scientific problem.

In [ ]:
torch.manual_seed(123)

lr_data = 1e-3
num_layers = 3
num_hidden = 32

model = FCN(1,2,num_hidden,num_layers)
optimizer = torch.optim.Adam(model.parameters(),lr=lr_data)

episodes = 6000
num_save_plot = round(episodes/60) # übernimmt 60 Bilder ins gif

files = []
name_plot = 'Inverted Pendulum NN: lr %s, hidden %sx%s'%(lr_data, num_layers, num_hidden)
name_gif = 'NN_lr_%s_hidden_%sx%s_episodes_%s.gif'%(lr_data, num_layers, num_hidden, episodes)

for i in range(1, episodes+1):

    optimizer.zero_grad()

    pred = model(t_data)

    loss = torch.mean((pred - data)**2) 

    loss.backward() 
    optimizer.step() 

    if (i) % num_save_plot == 0: 

        pred = model(t).detach()

        plot_my_result(t, x, theta, t_data, data, pred)

        file = "plots/nn_%.8i.png"%(i)
        plt.savefig(file, bbox_inches='tight', pad_inches=0.1, dpi=100, facecolor="white")
        files.append(file)

        if (i) % episodes == 0: plt.show()
        else: plt.close("all")

#save_gif_PIL(name_gif, files, fps=5, loop=0)       

## Physics Informed Neural Network

> Finally, we add the underlying differential equation ("physics loss") to the loss function. 

The physics loss aims to ensure that the learned solution is consistent with the underlying differential equation. This is done by penalising the residual of the differential equation over a set of locations sampled from the domain.

Here we evaluate the physics loss at 30 points uniformly spaced over the problem domain. We can calculate the derivatives of the network solution with respect to its input variable at these points using `pytorch`'s autodifferentiation features, and can then easily compute the residual of the differential equation using these quantities.

<div align="center">
  <a href="https://benmoseley.blog/my-research/so-what-is-a-physics-informed-neural-network/">
    <img src="https://benmoseley.blog/wp-content/uploads/2021/08/pinn.png" 
    alt="Images" width="600">
  </a>
</div>





>One way to do this for our problem is to use a physics-informed neural network [1,2]. The idea is very simple: add the known differential equations directly into the loss function when training the neural network.

>This is done by sampling a set of input training locations (\{x_{j}\}) and passing them through the network. Next gradients of the network’s output with respect to its input are computed at these locations (which are typically analytically available for most neural networks, and can be easily computed using autodifferentiation). Finally, the residual of the underlying differential equation is computed using these gradients, and added as an extra term in the loss function.

>The physics-informed neural network is able to predict the solution far away from the experimental data points, and thus performs much better than the naive network. One could argue that this network does indeed have some concept of our prior physical principles.

>The naive network is performing poorly because we are “throwing away” our existing scientific knowledge; with only the data at hand, it is like trying to understand all of the data generated by a particle collider, without having been to a physics class!

In [ ]:
import time
from torch import sin, cos

t_physics = torch.linspace(0,t1,t1*5).view(-1,1).requires_grad_(True)

torch.manual_seed(123)


lr_data = 1e-3
lr_physics = 1e-2
num_layers = 3
num_hidden = 32

model = FCN(1,2,num_hidden,num_layers)
optimizer = torch.optim.Adam(model.parameters(),lr=lr_data)


episodes = 4000
num_plots = 500                        


files = []
name_plot = 'Inverted Pendulum PINN: lr_data %s, lr_physics %s, hidden %sx%s'%(lr_data, lr_physics,num_layers, num_hidden)
name_gif = 'PINN_lr_data_%s_lr_physics_%s_hidden_%sx%s_episodes_%s.gif'%(lr_data, lr_physics,num_layers, num_hidden, episodes)


start = time.time()
for i in range(1, episodes+1):

    optimizer.zero_grad()

    # normal loss
    pred = model(t_data)
    loss_data = torch.mean((pred - data)**2) 

    # physics
    pred = model(t_physics)
    
    x_p = pred[:,0].view(-1,1)
    x_d_p = torch.autograd.grad(x_p, t_physics, torch.ones_like(x_p), create_graph=True)[0]
    x_dd_p = torch.autograd.grad(x_d_p, t_physics, torch.ones_like(x_d_p), create_graph=True)[0]
    
    theta_p = pred[:,1].view(-1,1)
    theta_d_p = torch.autograd.grad(theta_p, t_physics, torch.ones_like(theta_p), create_graph=True)[0]
    theta_dd_p = torch.autograd.grad(theta_d_p, t_physics, torch.ones_like(theta_d_p), create_graph=True)[0]
    
    sin_theta_p = sin(theta_p)
    cos_theta_p = cos(theta_p)
    
    physics_x = M*x_dd_p + m*(l*sin_theta_p*theta_d_p**2 - l*cos_theta_p*theta_dd_p + x_dd_p)
    physics_theta = l*m*(-g*sin_theta_p + l*theta_dd_p - cos_theta_p*x_dd_p)
    
    #physics = torch.cat((physics_x, physics_theta), 1)
    #loss_physics = lr_physics*torch.mean(physics**2)
    
    lambda_x = 0.02
    lambda_theta = 0.02
    loss_physics = lambda_x * torch.mean(physics_x**2) + lambda_theta * torch.mean(physics_theta**2)
    

    #loss = loss_data + loss_physics
    loss = loss_physics # ohne Loss durch die Messpunkte
    loss.backward() 
    optimizer.step() 
    
    # save 10 plots
    if (i % (episodes/num_plots)) == 0:   
        
        pred = model(t).detach()
        ps = t_physics.detach()
        
        plot_my_result(t, x, theta, t_data, data, pred, ps)
        
        file = "plots/pinn_%.8i.png"%(i)
        plt.savefig(file, bbox_inches='tight', pad_inches=0.1, dpi=100, facecolor="white")
        files.append(file)
        
        # show 5 plots
        if (i % (episodes/5)) == 0: plt.show()
        else: plt.close("all")
            

print('duration: %s [s]' % round(time.time() - start))            
save_gif_PIL(name_gif, files, fps=15, loop=0)  

# DeepXDE

In [ ]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch"""
import deepxde as dde
import numpy as np
from tensorflow import sin, cos


def ode_system_ip(t, y):
    
    x, theta = y[:, 0:1], y[:, 1:]
    
    x_d = dde.grad.jacobian(y, t, i=0)
    theta_d = dde.grad.jacobian(y, t, i=0)
    
    x_dd = dde.grad.hessian(y, t, component=0, i=0, j=0)
    theta_dd = dde.grad.hessian(y, t, component=1, i=0, j=0)
    
    sin_theta = sin(theta)
    cos_theta = cos(theta)
        
    #eq_x = M*x_dd+m*(l*tf.sin(theta)*theta_d**2-l*tf.cos(theta)*theta_dd+x_dd)
    eq_x = D*x_d + M*x_dd + m*(l*sin_theta*theta_d**2-l*cos_theta*theta_dd+x_dd)
    
    #eq_theta = l*m*(-g*tf.sin(theta)+l*theta_dd-tf.cos(theta)*x_dd)
    eq_theta = d*theta_d + l*m*(-g*sin_theta+l*theta_dd-cos_theta*x_dd)

    return [eq_x, eq_theta]


def boundary(_, on_initial):
    return on_initial

def func(t):

    return np.hstack((x_f(t), theta_f(t)))


geom = dde.geometry.TimeDomain(0, t1)
ic1 = dde.IC(geom, x_f, boundary, component=0)
ic2 = dde.IC(geom, theta_f, boundary, component=1)
data = dde.data.PDE(geom, ode_system_ip, [ic1, ic2], num_domain=50, num_boundary=2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [2]
activation = "sigmoid"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(epochs=20000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)